In [ ]:
import os
from os import listdir
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import re
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyheif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 22.6 MB/s eta 0:00:00


In [ ]:
# Define your dataset directory
Agriculture_crop = '/content/drive/MyDrive/crop_images'

# Define types
types = [
    "jute", "maize", "rice", "sugarcane", "wheat"
]

# Initialize lists to hold images and their labels
Agricultureimages = []
Agriculturelabels = []



target_size = (224, 224)  # Desired size to resize images

for i, AgricultureType in enumerate(types, start=1):
    AgricultureFolder = os.path.join(Agriculture_crop, str(i))
    for root, dirs, files in os.walk(AgricultureFolder):
        for file_name in files:
            img_path = os.path.join(root, file_name)
            _, file_extension = os.path.splitext(img_path)
            if file_extension.lower() == '.jpeg':
            #for add photo output folder
            #if file_extension.lower() == '.png':
                img = Image.open(img_path)
            elif file_extension.lower() == '.heic':
                try:
                    heif_file = pyheif.read(img_path)
                    img = Image.frombytes(
                        heif_file.mode,
                        heif_file.size,
                        heif_file.data,
                        "raw",
                        heif_file.mode,
                        heif_file.stride,
                    )
                except Exception as e:
                    print(f"Error opening {img_path}: {e}")
                    continue
            else:
                print(f"Unsupported file format: {img_path}")
                continue

            # Resize image to target size
            img = img.resize(target_size)

            # Append image and label
            Agricultureimages.append(np.array(img))
            Agriculturelabels.append(AgricultureType)

Agricultureimages = np.array(Agricultureimages)
Agriculturelabels = np.array(Agriculturelabels)

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

print("Number of images loaded:", len(Agricultureimages))
print("Number of labels:", len(Agriculturelabels))


Number of images loaded: 804
Number of labels: 804


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense


# Split the dataset into training and testing
train_images, test_images, train_labels, test_labels = train_test_split(
    Agricultureimages, Agriculturelabels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)
test_labels = label_binarizer.transform(test_labels)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(label_binarizer.classes_), activation="softmax"))
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with early stopping
history = model.fit(train_images, train_labels, epochs=100,
                    validation_data=(test_images, test_labels),
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/100
21/21 [==============================] - 894s 42s/step - loss: 441.0617 - accuracy: 0.2022 - val_loss: 1.6100 - val_accuracy: 0.1615
Epoch 2/100
21/21 [==============================] - 983s 47s/step - loss: 1.6092 - accuracy: 0.2208 - val_loss: 1.6088 - val_accuracy: 0.1739
Epoch 3/100
21/21 [==============================] - 899s 43s/step - loss: 1.6084 - accuracy: 0.2131 - val_loss: 1.6095 - val_accuracy: 0.1739
Epoch 4/100
21/21 [==============================] - 896s 42s/step - loss: 1.6063 - accuracy: 0.2006 - val_loss: 1.6073 - val_accuracy: 0.1739
Epoch 5/100
21/21 [==============================] - 891s 42s/step - loss: 1.5951 - accuracy: 0.2100 - val_loss: 1.5466 - val_accuracy: 0.2857
Epoch 6/100
21/21 [==============================] - 889s 43s/step - loss: 1.6943 - accuracy: 0.2006 - val_loss: 1.5976 - val_accuracy: 0.1925
Epoch 7/100
21/21 [==============================] - 877s 42s/step - loss: 1.5682 - accuracy: 0.2551 - val_loss: 1.5303 - val_accuracy: 0.29

In [ ]:
# IMAGE SIZE = 128,128
#--------- DATASET = 201
#1 32,64,128 -> 29
#2 32,32,64,64 WITH DROPOT(0.25)-> 24
#3 64,64,128,512 -> 31.70

#--------- DATASET = 804
#4 64,64,128,512 WITH DROPOT(0.25)-> 19
#5 64,64,128,128 WITH DROPOT(0.25)-> 16

# IMAGE SIZE = 224,224
#--------- DATASET = 804
#6 64,64,128,128 WITH DROPOT(0.25)-> 16
#7 32,64,128 -> 16
#new code
#8 32,32,64,64 WITH DROPOT(0.25)-> 42.85
#9 32,32,64,64,128,128 WITH DROPOT(0.25)->19.75

In [ ]:
import tensorflow as tf

# Assuming 'model' is your trained Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)